# Note
For continous training only run the "for cont'" cell

# Preparation

In [1]:
import os

MAIN_PATH = os.getcwd()
TRAINING_PATH = os.path.join(MAIN_PATH, "training")
DATA_PATH = os.path.join(MAIN_PATH, "data")
CONFIG_PATH = os.path.join(DATA_PATH, "pipeline.config")
LABEL_MAP_PATH = os.path.join(DATA_PATH, "label_map.pbtxt")
TRAIN_DATA_PATH = os.path.join(DATA_PATH, "train.record")
VAL_DATA_PATH = os.path.join(DATA_PATH, "validation.record")
TEST_DATA_PATH = os.path.join(DATA_PATH, "test.record")

print(TRAINING_PATH)

/root/train1/training


## Download the dataset

In [2]:
import requests

urls = ['https://dl.dropboxusercontent.com/s/9f5suk5oo5u7yyi/test.record',
        'https://dl.dropboxusercontent.com/s/tqvtgswib6r4f8u/validation.record',
        'https://dl.dropboxusercontent.com/s/6kgr2pwi7kgtmwb/train.record']
names = ["test.record",
        "validation.record",
        "train.record"]

for url, name in zip(urls, names):
    r = requests.get(url, allow_redirects=True)
    open(os.path.join(DATA_PATH, name), 'wb').write(r.content)

## Clone the TensorFlow Model Garden

In [3]:
# (For cont')
%cd {MAIN_PATH}
!git clone https://github.com/ahmadnurokhim/models.git


/root/train1
Cloning into 'models'...
remote: Enumerating objects: 74201, done.
remote: Counting objects: 100% (287/287), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 74201 (delta 129), reused 235 (delta 97), pack-reused 73914
Receiving objects: 100% (74201/74201), 580.22 MiB | 30.60 MiB/s, done.
Resolving deltas: 100% (52568/52568), done.


## TF Object Detection API

### Compile/install the TF Object Detection API

CD to the models/research

In [4]:
# (For cont')
%cd {MAIN_PATH}/models/research

/root/train1/models/research


### Install the protoc files

In [5]:
!apt install protobuf-compiler -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
The following NEW packages will be installed:
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
  protobuf-compiler
0 upgraded, 5 newly installed, 0 to remove and 15 not upgraded.
Need to get 2758 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotobuf-lite17 amd64 3.6.1.3-2ubuntu5 [132 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotobuf17 amd64 3.6.1.3-2ubuntu5 [798 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotoc17 amd64 3.6.1.3-2ubuntu5 [646 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libprotobuf-dev amd64 3.6.1.3-2ubuntu5 [1155 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 protobuf-compiler a

In [6]:
# (For cont')

!protoc object_detection/protos/*.proto --python_out=.

(For cont') \
\
To avoid DNN library not found problem, open the object_detection/packages/tf2/setup.py and make sure to add these lines or you can just run the next cell
```
    'tensorflow==2.8.0',
    'tf-models-official==2.8.0',
    'tensorflow_io==0.23.1',
    'keras==2.8.0',
```



In [7]:
#!pip install tensorflow==2.8.0
#!pip install tf-models-official==2.8.0
#!pip install tensorflow_io==0.23.1
#!pip install keras==2.8.0
!pip install tf-models-official
!pip install tensorflow_io
!pip install keras

     |████████████████████████████████| 2.1 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 7.6 MB/s  eta 0:00:01
     |████████████████████████████████| 108 kB 66.0 MB/s eta 0:00:01
     |████████████████████████████████| 106 kB 67.2 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.6 MB 38.8 MB/s eta 0:00:01
     |████████████████████████████████| 662 kB 51.3 MB/s eta 0:00:01
     |████████████████████████████████| 41.6 MB 54.9 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 2.8 MB/s  eta 0:00:01
     |████████████████████████████████| 11.7 MB 39.9 MB/s eta 0:00:01
     |████████████████████████████████| 352 kB 50.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 55.6 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 12.5 MB/s eta 0:00:01
  Using cached Cython-0.29.30-cp38-cp38-manylinu

     |████████████████████████████████| 95 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 98 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 142 kB 61.4 MB/s eta 0:00:01
     |████████████████████████████████| 764 kB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 7.6 MB/s  eta 0:00:01
     |████████████████████████████████| 114 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 56.4 MB/s eta 0:00:01
     |████████████████████████████████| 930 kB 57.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 8.6 MB/s  eta 0:00:01
     |████████████████████████████████| 306 kB 53.8 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 53.2 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73032 sha256=da27424fed96d4e0cf564a61b19139818cb5fef13f9f0df743796551126f7a92
  Stored in directory: /ro

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 25.9 MB 2.3 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
# (For cont')

!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /root/train1/models/research
     |████████████████████████████████| 12.3 MB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 59.6 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 17.1 MB/s eta 0:00:01
     |████████████████████████████████| 26.7 MB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 89 kB 23.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 46.2 MB/s eta 0:00:01
     |████████████████████████████████| 546 kB 50.0 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 95 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 256 kB 53.2 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 62.7 MB/s eta 0:00:01
     |████████████████████████████████| 60.5 MB 52.8 MB/s eta 0:00:01


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1693568 sha256=298770672daff4291d9e30abc15ed55eb39c0967e952a30cb5b2ed97128389d0
  Stored in directory: /tmp/pip-ephem-wheel-cache-m9juhbqr/wheels/63/a7/0e/470621870b3a152ee838e90ee41621c4a103254383f3be3d97
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43992 sha256=d7aec39c848ded726166e1936634a55eeae27cd429f0a7c7954314139f95795a
  Stored in directory: /root/.cache/pip/wheels/bb/73/e9/d273421f5723c4bf544dcf9eb097bda94421ef8d3252699f0a
  Created wheel for crcmod: filename=crcmod-1.7-cp38-cp38-linux_x86_64.whl size=36003 sha256=a217ab998d5e1e0d3b628868697c4042ce5cbf83772e890a57c1feb1974e0184
  Stored in directory: /root/.cache/pip/wheels/ca/5a/02/f3acf982a026f3319fb3e798a8dca2d48fafee7761788562e9
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78543 sha256=0ed930910737ac542d2e50d794678e2ac0c18fd9810a45e852d3315c7ac2b68a
  Stored in directory

### Test the TF Object Detection installation API

In [9]:
# (For cont')

!python object_detection/builders/model_builder_tf2_test.py

Running tests under Python 3.8.10: /usr/local/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-06-04 12:15:55.571735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-04 12:15:56.725578: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22340 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-06-04 12:15:56.727285: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22340 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6
/usr/local/lib/python3.8/

I0604 12:16:00.762233 140019516405568 efficientnet_model.py:143] round_filter input=16 output=16
I0604 12:16:00.762325 140019516405568 efficientnet_model.py:143] round_filter input=24 output=24
I0604 12:16:00.875456 140019516405568 efficientnet_model.py:143] round_filter input=24 output=24
I0604 12:16:00.875553 140019516405568 efficientnet_model.py:143] round_filter input=40 output=40
I0604 12:16:00.989835 140019516405568 efficientnet_model.py:143] round_filter input=40 output=40
I0604 12:16:00.989939 140019516405568 efficientnet_model.py:143] round_filter input=80 output=80
I0604 12:16:01.162424 140019516405568 efficientnet_model.py:143] round_filter input=80 output=80
I0604 12:16:01.162549 140019516405568 efficientnet_model.py:143] round_filter input=112 output=112
I0604 12:16:01.335368 140019516405568 efficientnet_model.py:143] round_filter input=112 output=112
I0604 12:16:01.335484 140019516405568 efficientnet_model.py:143] round_filter input=192 output=192
I0604 12:16:01.573688 14

I0604 12:16:04.139407 140019516405568 efficientnet_model.py:143] round_filter input=112 output=120
I0604 12:16:04.139548 140019516405568 efficientnet_model.py:143] round_filter input=192 output=208
I0604 12:16:04.439208 140019516405568 efficientnet_model.py:143] round_filter input=192 output=208
I0604 12:16:04.439340 140019516405568 efficientnet_model.py:143] round_filter input=320 output=352
I0604 12:16:04.555382 140019516405568 efficientnet_model.py:143] round_filter input=1280 output=1408
I0604 12:16:04.578605 140019516405568 efficientnet_model.py:453] Building model efficientnet with params ModelConfig(width_coefficient=1.1, depth_coefficient=1.2, resolution=260, dropout_rate=0.3, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

I0604 12:16:08.431102 140019516405568 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet EfficientNet backbone version: efficientnet-b5
I0604 12:16:08.431291 140019516405568 ssd_efficientnet_bifpn_feature_extractor.py:147] EfficientDet BiFPN num filters: 288
I0604 12:16:08.431373 140019516405568 ssd_efficientnet_bifpn_feature_extractor.py:148] EfficientDet BiFPN num iterations: 7
I0604 12:16:08.432895 140019516405568 efficientnet_model.py:143] round_filter input=32 output=48
I0604 12:16:08.445641 140019516405568 efficientnet_model.py:143] round_filter input=32 output=48
I0604 12:16:08.445722 140019516405568 efficientnet_model.py:143] round_filter input=16 output=24
I0604 12:16:08.586113 140019516405568 efficientnet_model.py:143] round_filter input=16 output=24
I0604 12:16:08.586246 140019516405568 efficientnet_model.py:143] round_filter input=24 output=40
I0604 12:16:08.876193 140019516405568 efficientnet_model.py:143] round_filter input=24 output=40
I0604 12:16:08.876351 14

I0604 12:16:13.899262 140019516405568 efficientnet_model.py:143] round_filter input=16 output=32
I0604 12:16:13.899398 140019516405568 efficientnet_model.py:143] round_filter input=24 output=48
I0604 12:16:14.313197 140019516405568 efficientnet_model.py:143] round_filter input=24 output=48
I0604 12:16:14.313328 140019516405568 efficientnet_model.py:143] round_filter input=40 output=80
I0604 12:16:14.730289 140019516405568 efficientnet_model.py:143] round_filter input=40 output=80
I0604 12:16:14.730424 140019516405568 efficientnet_model.py:143] round_filter input=80 output=160
I0604 12:16:15.314172 140019516405568 efficientnet_model.py:143] round_filter input=80 output=160
I0604 12:16:15.314306 140019516405568 efficientnet_model.py:143] round_filter input=112 output=224
I0604 12:16:15.894876 140019516405568 efficientnet_model.py:143] round_filter input=112 output=224
I0604 12:16:15.895009 140019516405568 efficientnet_model.py:143] round_filter input=192 output=384
I0604 12:16:16.653286 

## Download pre-trained model checkpoint

In [10]:
%cd {DATA_PATH}

/root/train1/data


In [ ]:
!apt install wget -y

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

## Modify the pipeline.config (IMPORTANT!)
change the checkpoint, label map, train, and test in data/pipeline.config to below path 

In [11]:
print("Checkpoint: " + os.path.join(DATA_PATH, "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8", "checkpoint", "ckpt-0"))
print("Label map: " + LABEL_MAP_PATH)
print("Train: " + TRAIN_DATA_PATH)
print("Test: " + TEST_DATA_PATH)


Checkpoint: /root/train1/data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0
Label map: /root/train1/data/label_map.pbtxt
Train: /root/train1/data/train.record
Test: /root/train1/data/test.record


Also change add "max_to_keep=None" as parameter in tf.train.Saver() in /research/object_detection/legacy/trainer.py \
So it'll be:\
\
saver = tf.train.Saver(max_to_keep=None, \
keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours)


## Load Tensorboard

In [ ]:
# (For cont')

%load_ext tensorboard
%tensorboard --logdir /root/train1/training

# Train The Model

## Fixing incompatible libraries

In [12]:
# (For cont')
#==4.5.4.60
!pip uninstall opencv-python --y
!pip uninstall opencv-contrib-python --y
!pip uninstall opencv-python-headless --y
!pip install opencv-python==4.5.5.64 
!pip install opencv-contrib-python==4.5.5.64 
!pip install opencv-python-headless==4.5.5.64 

Found existing installation: opencv-python 4.5.5.64
Uninstalling opencv-python-4.5.5.64:
  Successfully uninstalled opencv-python-4.5.5.64
Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Successfully uninstalled opencv-python-headless-4.5.5.64
  Using cached opencv_python-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.5 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 66.7 MB 153 kB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
  Using cached opencv_python_headless-4.5.5.64-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (47.8 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


## Training

In [ ]:
# (For cont')

%cd {MAIN_PATH}/models/research/object_detection

!python model_main_tf2.py \
--pipeline_config_path={CONFIG_PATH} \
--model_dir={TRAINING_PATH} \
--alsologtostderr

/root/train1/models/research/object_detection
2022-06-04 13:46:36.873828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-04 13:46:37.702119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22340 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:01:00.0, compute capability: 8.6
2022-06-04 13:46:37.703288: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22340 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:82:00.0, compute capability: 8.6
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',

INFO:tensorflow:batch_all_reduce: 232 all-reduces with algorithm = nccl, num_packs = 1
I0604 13:47:38.168219 139845714130752 cross_device_ops.py:897] batch_all_reduce: 232 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0604 13:47:40.404885 139845714130752 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0604 13:47:40.408127 139845714130752 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/de

INFO:tensorflow:Step 36000 per-step time 0.377s
I0604 13:55:08.196836 139845714130752 model_lib_v2.py:705] Step 36000 per-step time 0.377s
INFO:tensorflow:{'Loss/classification_loss': 0.08683649,
 'Loss/localization_loss': 0.050683074,
 'Loss/regularization_loss': 0.084303446,
 'Loss/total_loss': 0.221823,
 'learning_rate': 0.015060408}
I0604 13:55:08.197113 139845714130752 model_lib_v2.py:708] {'Loss/classification_loss': 0.08683649,
 'Loss/localization_loss': 0.050683074,
 'Loss/regularization_loss': 0.084303446,
 'Loss/total_loss': 0.221823,
 'learning_rate': 0.015060408}
INFO:tensorflow:Step 36100 per-step time 0.378s
I0604 13:55:46.024338 139845714130752 model_lib_v2.py:705] Step 36100 per-step time 0.378s
INFO:tensorflow:{'Loss/classification_loss': 0.088408165,
 'Loss/localization_loss': 0.054447204,
 'Loss/regularization_loss': 0.08422506,
 'Loss/total_loss': 0.22708043,
 'learning_rate': 0.014860413}
I0604 13:55:46.024629 139845714130752 model_lib_v2.py:708] {'Loss/classificat

INFO:tensorflow:{'Loss/classification_loss': 0.1006263,
 'Loss/localization_loss': 0.076978244,
 'Loss/regularization_loss': 0.08326079,
 'Loss/total_loss': 0.26086533,
 'learning_rate': 0.012357492}
I0604 14:03:52.872341 139845714130752 model_lib_v2.py:708] {'Loss/classification_loss': 0.1006263,
 'Loss/localization_loss': 0.076978244,
 'Loss/regularization_loss': 0.08326079,
 'Loss/total_loss': 0.26086533,
 'learning_rate': 0.012357492}
INFO:tensorflow:Step 37500 per-step time 0.375s
I0604 14:04:30.359035 139845714130752 model_lib_v2.py:705] Step 37500 per-step time 0.375s
INFO:tensorflow:{'Loss/classification_loss': 0.079909235,
 'Loss/localization_loss': 0.04955727,
 'Loss/regularization_loss': 0.08319079,
 'Loss/total_loss': 0.2126573,
 'learning_rate': 0.012172694}
I0604 14:04:30.359259 139845714130752 model_lib_v2.py:708] {'Loss/classification_loss': 0.079909235,
 'Loss/localization_loss': 0.04955727,
 'Loss/regularization_loss': 0.08319079,
 'Loss/total_loss': 0.2126573,
 'lear

## Validation

In [ ]:
!python model_main_tf2.py \
--model_dir={TRAINING_PATH} \
--pipeline_config_path={CONFIG_PATH} \
--checkpoint_dir={TRAINING_PATH}

## Inference/Save The Model

In [ ]:
!python exporter_main_v2.py \
--trained_checkpoint_dir={TRAINING_PATH} \
--pipeline_config_path={CONFIG_PATH} \
--output_directory {DATA_PATH}/inference_graph